In [23]:
import numpy as np
import pandas as pd
from IPython.display import display

In [24]:
data_input = pd.read_excel("E:/Graduate_Project/IEEE ISI World Cup 2019/competition1/test/债券信息.xlsx")
data_input.rename(columns={'企业总评分': '企业编号'}, inplace=True)  # 修改列名

In [25]:
display(data_input.head())
m = data_input.shape[0]
print(data_input['债券期限'].isnull().sum())
data_input['债券期限'].fillna(method='pad', inplace=True)  # 缺失值向前填充

,企业编号,债券信用评级,债券期限,债券品种,发行日期,付息日期,兑付日期,计划发行总额（亿元）,实际发行总额（亿元）,币种,流通场所,票面利率（%）,主体信用评级,付息方式
0,1002,NaN,0.49,银行间企债,2018-04-13,04-16,2018-10-13,5.0,NaN,人民币,NaN,4.35,NaN,到期一次还本付息
1,1002,NaN,0.49,银行间企债,2018-04-17,04-18,2018-10-15,10.0,NaN,人民币,NaN,4.35,NaN,到期一次还本付息
2,1002,NaN,0.49,银行间企债,2018-04-20,04-23,2018-10-20,3.0,NaN,人民币,NaN,4.35,NaN,到期一次还本付息
3,1002,NaN,0.49,银行间企债,2018-09-27,09-28,2019-03-27,10.0,NaN,人民币,NaN,3.05,NaN,到期一次还本付息
4,1002,NaN,0.49,银行间企债,2018-10-08,10-10,2019-04-08,5.0,NaN,人民币,NaN,3.05,NaN,到期一次还本付息


86


In [26]:
for i in range(m):
    if '年' in data_input['债券期限'][i]:
        data_input['债券期限'][i] = data_input['债券期限'][i][:-1]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
data_input['债券期限'] = data_input['债券期限'].astype(np.float)
print(data_input['债券期限'])

0        0.49
1        0.49
2        0.49
3        0.49
4        0.49
5        0.49
6        5.00
7        3.00
8       10.00
9        0.74
10       5.00
11       5.00
12       3.00
13       3.00
14       3.00
15       3.00
16       3.00
17       0.74
18       0.47
19       0.74
20       3.00
21      10.00
22       3.00
23       3.00
24       7.00
25      10.00
26      10.00
27       5.00
28      10.00
29       0.74
        ...  
2370     3.00
2371     3.00
2372     5.00
2373     3.00
2374     3.00
2375     4.00
2376     4.00
2377     5.00
2378     0.74
2379     5.00
2380     0.74
2381     5.00
2382     3.00
2383     3.00
2384     3.00
2385     3.00
2386     0.74
2387     3.00
2388     3.00
2389     5.00
2390     5.00
2391     0.96
2392     5.00
2393     5.00
2394     5.00
2395     5.00
2396     3.00
2397     3.00
2398     3.00
2399     5.00
Name: 债券期限, Length: 2400, dtype: float64


In [28]:
data_input.drop(['发行日期', '付息日期', '兑付日期', '币种', '流通场所'], axis=1, inplace=True)

In [29]:
cl = pd.Series(name='期限类别', index=range(m))
for i in range(m):
    if data_input['债券期限'][i] < 1.00:
        cl[i] = '短'
    elif data_input['债券期限'][i] >= 10.00:
        cl[i] = '长'
    else:
        cl[i] = '中'
print(cl.head())

0    短
1    短
2    短
3    短
4    短
Name: 期限类别, dtype: object


In [30]:
data_input['债券期限'] = cl
data1 = data_input.loc[:,['企业编号','债券信用评级','债券期限','债券品种','计划发行总额（亿元）']] #取多列
data1.head()

,企业编号,债券信用评级,债券期限,债券品种,计划发行总额（亿元）
0,1002,NaN,短,银行间企债,5.0
1,1002,NaN,短,银行间企债,10.0
2,1002,NaN,短,银行间企债,3.0
3,1002,NaN,短,银行间企债,10.0
4,1002,NaN,短,银行间企债,5.0


In [31]:
index = data_input.index
feature_names = []
for feature in ['银行间企债','中央企业债','地方企业债','公司、企业债','沪企债','深企债','企业债券']:
    for length in ['短','中','长']:
        feature_names.append(feature+':'+length)
col_names = data1.columns.tolist() + feature_names
col_names

['企业编号',
 '债券信用评级',
 '债券期限',
 '债券品种',
 '计划发行总额（亿元）',
 '银行间企债:短',
 '银行间企债:中',
 '银行间企债:长',
 '中央企业债:短',
 '中央企业债:中',
 '中央企业债:长',
 '地方企业债:短',
 '地方企业债:中',
 '地方企业债:长',
 '公司、企业债:短',
 '公司、企业债:中',
 '公司、企业债:长',
 '沪企债:短',
 '沪企债:中',
 '沪企债:长',
 '深企债:短',
 '深企债:中',
 '深企债:长',
 '企业债券:短',
 '企业债券:中',
 '企业债券:长']

In [32]:
data1 = data1.reindex(index=index, columns=col_names, fill_value=0)
data1.head()

,企业编号,债券信用评级,债券期限,债券品种,计划发行总额（亿元）,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:短,中央企业债:中,...,公司、企业债:长,沪企债:短,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:短,企业债券:中,企业债券:长
0,1002,NaN,短,银行间企债,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,NaN,短,银行间企债,10.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1002,NaN,短,银行间企债,3.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1002,NaN,短,银行间企债,10.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1002,NaN,短,银行间企债,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
for feature in ['银行间企债','中央企业债','地方企业债','公司、企业债','沪企债','深企债','企业债券']:
    for length in ['短','中','长']:
        data1.loc[(data1['债券期限']==length) & (data1['债券品种']==feature), feature+':'+length]=data1['计划发行总额（亿元）']
data1

,企业编号,债券信用评级,债券期限,债券品种,计划发行总额（亿元）,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:短,中央企业债:中,...,公司、企业债:长,沪企债:短,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:短,企业债券:中,企业债券:长
0,1002,NaN,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,1002,NaN,短,银行间企债,10.00,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,1002,NaN,短,银行间企债,3.00,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,1002,NaN,短,银行间企债,10.00,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,1002,NaN,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,1002,NaN,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,1004,AAA,中,公司、企业债,5.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,1004,AAA,中,公司、企业债,4.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,1004,AAA,长,中央企业债,10.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,1004,NaN,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [34]:
data1['债券信用评级'] = data1['债券信用评级'].fillna(0)  # 缺失值补零
levels = ['C', 'CC', 'B', 'AA-', 'AA', 'AA+', 'AAA']
for i in range(len(levels)):
    data1.loc[data1['债券信用评级'] == levels[i], '债券信用评级'] = i+1
data1

,企业编号,债券信用评级,债券期限,债券品种,计划发行总额（亿元）,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:短,中央企业债:中,...,公司、企业债:长,沪企债:短,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:短,企业债券:中,企业债券:长
0,1002,0,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,1002,0,短,银行间企债,10.00,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,1002,0,短,银行间企债,3.00,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,1002,0,短,银行间企债,10.00,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,1002,0,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,1002,0,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,1004,7,中,公司、企业债,5.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,1004,7,中,公司、企业债,4.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,1004,7,长,中央企业债,10.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,1004,0,短,银行间企债,5.00,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [35]:
data1.drop(['债券期限', '债券品种', '计划发行总额（亿元）'], axis=1, inplace=True)
data1

,企业编号,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:短,中央企业债:中,中央企业债:长,地方企业债:短,地方企业债:中,...,公司、企业债:长,沪企债:短,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:短,企业债券:中,企业债券:长
0,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,1002,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,1002,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,1002,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,1004,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,1004,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,1004,7,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,1004,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [36]:
data1 = data1.loc[:, (data1 != 0).any(axis = 0)]
data1

,企业编号,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:中,中央企业债:长,地方企业债:中,地方企业债:长,公司、企业债:中,公司、企业债:长,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:中,企业债券:长
0,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,1002,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,1002,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,1002,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,1002,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,1004,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,1004,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,1004,7,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,1004,0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [37]:
data1 = data1.groupby('企业编号').sum()
data1 = data1.reset_index()
data1

,企业编号,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:中,中央企业债:长,地方企业债:中,地方企业债:长,公司、企业债:中,公司、企业债:长,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:中,企业债券:长
0,1002,0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,1004,21,5.0,5.0,0.0,0.0,10.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,1005,7,20.0,40.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,1006,0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,1007,0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.00,0.0,0.0,0.0
5,1008,14,285.0,170.0,0.0,0.0,0.0,0.0,0.0,30.0,12.0,88.0,123.0,0.0,0.00,0.0,0.0,0.0
6,1012,0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.00,0.0,0.0,0.0
7,1013,21,40.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,1014,0,140.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,1015,7,32.0,22.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,48.0,0.0,0.0,0.00,0.0,0.0,0.0


In [38]:
# data1.to_excel('E:/Graduate_Project/competition1/after/债券信息_特征.xlsx')

In [42]:
data2 = pd.read_excel('E:/Graduate_Project/IEEE ISI World Cup 2019/competition1/yyx/企业评分.xlsx')
data_out = pd.merge(data2, data1, how='left')
display(data_out)

,企业编号,企业总评分,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:中,中央企业债:长,地方企业债:中,地方企业债:长,公司、企业债:中,公司、企业债:长,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:中,企业债券:长
0,1001,75.374276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,79.830122,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,78.318264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,83.253376,21.0,5.0,5.0,0.0,0.0,10.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005,83.291493,7.0,20.0,40.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1006,82.013621,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1007,87.675653,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1008,79.592075,14.0,285.0,170.0,0.0,0.0,0.0,0.0,0.0,30.0,12.0,88.0,123.0,0.0,0.0,0.0,0.0,0.0
8,1009,78.236148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1010,80.895705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
data_out.fillna(0, inplace=True)
display(data_out)

,企业编号,企业总评分,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:中,中央企业债:长,地方企业债:中,地方企业债:长,公司、企业债:中,公司、企业债:长,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:中,企业债券:长
0,1001,75.374276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1002,79.830122,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,78.318264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1004,83.253376,21.0,5.0,5.0,0.0,0.0,10.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005,83.291493,7.0,20.0,40.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1006,82.013621,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1007,87.675653,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1008,79.592075,14.0,285.0,170.0,0.0,0.0,0.0,0.0,0.0,30.0,12.0,88.0,123.0,0.0,0.0,0.0,0.0,0.0
8,1009,78.236148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1010,80.895705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
data_out.drop('企业总评分', axis=1, inplace=True)
data_out.drop_duplicates(inplace=True)
data_out

,企业编号,债券信用评级,银行间企债:短,银行间企债:中,银行间企债:长,中央企业债:中,中央企业债:长,地方企业债:中,地方企业债:长,公司、企业债:中,公司、企业债:长,沪企债:中,沪企债:长,深企债:短,深企债:中,深企债:长,企业债券:中,企业债券:长
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1002,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1004,21.0,5.0,5.0,0.0,0.0,10.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005,7.0,20.0,40.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1006,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1007,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1008,14.0,285.0,170.0,0.0,0.0,0.0,0.0,0.0,30.0,12.0,88.0,123.0,0.0,0.0,0.0,0.0,0.0
8,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
data_out.to_excel('E:/Graduate_Project/IEEE ISI World Cup 2019/competition1/after/债券信息_特征.xlsx', index=False)